In [5]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.document_loaders.text import TextLoader
from pydantic import BaseModel ,Field
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from dotenv import dotenv_values
import json
import pandas as pd



# Load "gemini-pro"

In [6]:
env_values = dotenv_values(".env")
Gimni_API_KEY = env_values['Gimni_API_KEY']

In [7]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=Gimni_API_KEY,temperature=0.5)

# Load Extracted Text 

In [8]:
loader = TextLoader('Tesseract_output/output_text/Enviromental_factors_egypt_arabic/all_text.txt')
all_text=loader.load()

In [9]:
print(all_text[0].page_content)

مصرء أرض الأهرامات الشاهقة والعجائب القديمة» تتمتع ببيئة فريدة تشكلت من خلال التفاعل المعقد بين العوامل الحية
(الحيوية) والعوامل غير الحية (اللاأحيائية). فهم هذه العوامل ضروري لتقدير التوازن البيئي الدقيق لمصر والتحديات التي
.تواجهها

أرض التناقضات: وفرة حيوية وجفاف

على الرغم من المناخ الصحراوي القاسيء تدعم مصر مجموعة متنوعة من الحياة النباتية والحيوانية. تتكيف أشجار الأكاسيا
بقدرتها على حفظ المياه مع الرمال الصحراويةء بينما تنمو نباتات البردي ونخيل التمر على طول نهر النيل الذي يمنح الحياة»
مما يوفر الغذاء والمأوى لمجموعة متنوعة من الحيوانات. هذه النظم البيئية النابضة بالحياة تظهر مرونة الحياة الرائعة في
.مصر

نهر النيل: شريان الحياة لمصر

نهر النيل هو قلب النظام البيئي لمصرء شريان حيوي حافظ على السكان والزراعة لآلاف السنين. كانت فيضاناته السنوية تودع
الطمي الخصيب. مما يخلق دلتا النيل» وهي منطقة زراعية حيوية. هذه الشريط الخصيب من الأرض وسط الصحراء الشاسعة
يبرز أهمية النيل للحياة في مصر. ومع ذلك. فإن بناء السدود غيّر الدورة الفيضانية الطبيعية» مما أثر على تجديد التربة
.الخصبة وتقليل تد

# Split Text into Chunks

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    # is_separator_regex=False,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)

In [11]:
query_1="Enviromental_factors_egypt_arabic"
meta_data={"document":query_1}
chunks = text_splitter.create_documents([all_text[0].page_content],metadatas=[meta_data])


In [12]:
len(chunks)

3

In [13]:
print (chunks[1].page_content)

يؤدي تغير المناخ إلى اضطراب الأنماط الجوية» مما يؤثر على التوازن الدقيق لأنظمة مصر البيئية. يمكن أن تؤدي الزيادة في
درجات الحرارة العالمية إلى تفاقم التصحرء مما يقلل من إنتاجية الزراعة. بالإضافة إلى ذلكء» يمكن أن تؤدي التغيرات في
أنماط هطول الأمطار إلى اضطراب الموارد المائية المحدودة بالفعل في البلاد. يمكن أيضًا للأنشطة البشرية مثل الرعي الجائر
.والممارسات الزراعية غير المستدامة أن تهدد التوازن الدقيق لأنظمة الصحراء البيئية وتقلل من الأراضي الصالحة للزراعة
الكشف عن عظمة النيل


نهر النيل العظيم: الكشف عن عظمته بالأرقام

يمتد نهر النيل على طول 6,650 كيلومترًا (4,132 ميلا)» وهو أطول نهر في إفريقيا. بمتوسط تدفق يبلغ 2,830 متر مكعب
في الثانية (99,500 قدم مكعب في الثانية)» يعد مصدرً! رئيسيًا للمياه العذبة لمصر. دلتا النيل» التي تشكلت من ترسبات النهر
على مدى آلاف السنين؛ تمتد على حوالي 24,000 كيلومتر مربع (9,300 ميل مربع)؛ وهي منطقة خصبة حيوية للزراعة.
ومع ذلكء فإن بناء السدود على طول النيل غيّر الدورة الفيضانية الطبيعية» مما أتر على تجديد التربة الخصبة وقد يعرقل توافر
.المياه في المستقبل بس

# Create output parser for the prompt

In [14]:
class pairs(BaseModel):
    
    KPI: str = Field(description="A KPI identified within a text.")
    context: str = Field(description="An Arabic context that can be used to extract the identified KPIs from a text.")

In [15]:
# Set up a parser + inject instructions into the prompt template.
# parser = JsonOutputParser(pydantic_object=pairs)
parser=PydanticOutputParser(pydantic_object=pairs)

In [16]:
temp="\n".join([

    "قم بتقسيم النص العربي التالي إلى جمل واستخرج مؤشرات الأداء الرئيسية (KPIs) من كل جملة.", 
   "يجب أن تتضمن مؤشرات الأداء الرئيسية بيانات رقمية، أو مقاييس أداء، أو معلومات إحصائية تتعلق بالعوامل البيئية. ",
   "إذا لم تكن مؤشرات الأداء الرئيسية واردة في الجملة ،تخطي هذة الجملة'"
   

    "### text:",
    "{text}",
    "{format_instructions}",
])

prompt=PromptTemplate(
    template=temp,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [19]:
result_all=[]
for chunk in chunks:
    message_all = prompt.format(text=chunk.page_content)
    result=llm( message_all )
    result_all.append(result.split("\n"))

In [20]:
result_all

[['```json',
  '{"KPI": null, "context": "مصرء أرض الأهرامات الشاهقة والعجائب القديمة» تتمتع ببيئة فريدة تشكلت من خلال التفاعل المعقد بين العوامل الحية\\n(الحيوية) والعوامل غير الحية (اللاأحيائية). فهم هذه العوامل ضروري لتقدير التوازن البيئي الدقيق لمصر والتحديات التي\\n.تواجهها"}',
  '{"KPI": null, "context": "أرض التناقضات: وفرة حيوية وجفاف\\nعلى الرغم من المناخ الصحراوي القاسيء تدعم مصر مجموعة متنوعة من الحياة النباتية والحيوانية. تتكيف أشجار الأكاسيا\\nبقدرتها على حفظ المياه مع الرمال الصحراويةء بينما تنمو نباتات البردي ونخيل التمر على طول نهر النيل الذي يمنح الحياة»\\nMما يوفر الغذاء والمأوى لمجموعة متنوعة من الحيوانات. هذه النظم البيئية النابضة بالحياة تظهر مرونة الحياة الرائعة في\\n.مصر"}',
  '{"KPI": null, "context": "نهر النيل: شريان الحياة لمصر\\nنهر النيل هو قلب النظام البيئي لمصرء شريان حيوي حافظ على السكان والزراعة لآلاف السنين. كانت فيضاناته السنوية تودع\\nالطمي الخصيب. مما يخلق دلتا النيل» وهي منطقة زراعية حيوية. هذه الشريط الخصيب من الأرض وسط الصحراء الشاسعة\\nيبرز أهمي

In [21]:


json_objects = []

for block in result_all:
    for line in block:
        line = line.strip()
        if line.startswith('{') and line.endswith('}'):
            json_object = json.loads(line)
            json_objects.append(json_object)

filtered_data = [entry for entry in json_objects if entry["KPI"] is not None]
output_json = json.dumps(filtered_data, ensure_ascii=False, indent=2)
print(output_json)

[
  {
    "KPI": "6,650 كيلومترًا (4,132 ميلا)",
    "context": "يمتد نهر النيل على طول 6,650 كيلومترًا (4,132 ميلا)» وهو أطول نهر في إفريقيا."
  },
  {
    "KPI": "2,830 متر مكعب في الثانية (99,500 قدم مكعب في الثانية)",
    "context": "بمتوسط تدفق يبلغ 2,830 متر مكعب في الثانية (99,500 قدم مكعب في الثانية)» يعد مصدرً! رئيسيًا للمياه العذبة لمصر."
  },
  {
    "KPI": "24,000 كيلومتر مربع (9,300 ميل مربع)",
    "context": "دلتا النيل» التي تشكلت من ترسبات النهر على مدى آلاف السنين؛ تمتد على حوالي 24,000 كيلومتر مربع (9,300 ميل مربع)؛ وهي منطقة خصبة حيوية للزراعة."
  },
  {
    "KPI": "9.4 مليون كيلومتر مربع (3.6 مليون ميل مربع)",
    "context": "تغطي الصحراء الكبرى» التي تشمل جزءًا كبيرًا من مساحة مصرء بيئة قاسية مع ندرة في الأمطار ودرجات حرارة متطرفة. هذه الصحراء الشاسعة» التي تعد أكبر صحراء حارة في العالم» تغطي حوالي 9.4 مليون كيلومتر مربع (3.6 مليون ميل مربع)."
  },
  {
    "KPI": "أقل من 250 مليمترًا (9.8 بوصة)",
    "context": "يبلغ متوسط هطول الأمطار السنوي في الصحراء أقل من 250 

In [22]:
filtered_data

[{'KPI': '6,650 كيلومترًا (4,132 ميلا)',
  'context': 'يمتد نهر النيل على طول 6,650 كيلومترًا (4,132 ميلا)» وهو أطول نهر في إفريقيا.'},
 {'KPI': '2,830 متر مكعب في الثانية (99,500 قدم مكعب في الثانية)',
  'context': 'بمتوسط تدفق يبلغ 2,830 متر مكعب في الثانية (99,500 قدم مكعب في الثانية)» يعد مصدرً! رئيسيًا للمياه العذبة لمصر.'},
 {'KPI': '24,000 كيلومتر مربع (9,300 ميل مربع)',
  'context': 'دلتا النيل» التي تشكلت من ترسبات النهر على مدى آلاف السنين؛ تمتد على حوالي 24,000 كيلومتر مربع (9,300 ميل مربع)؛ وهي منطقة خصبة حيوية للزراعة.'},
 {'KPI': '9.4 مليون كيلومتر مربع (3.6 مليون ميل مربع)',
  'context': 'تغطي الصحراء الكبرى» التي تشمل جزءًا كبيرًا من مساحة مصرء بيئة قاسية مع ندرة في الأمطار ودرجات حرارة متطرفة. هذه الصحراء الشاسعة» التي تعد أكبر صحراء حارة في العالم» تغطي حوالي 9.4 مليون كيلومتر مربع (3.6 مليون ميل مربع).'},
 {'KPI': 'أقل من 250 مليمترًا (9.8 بوصة)',
  'context': 'يبلغ متوسط هطول الأمطار السنوي في الصحراء أقل من 250 مليمترًا (9.8 بوصة).؛ مما يخلق بيئة صعبة لمعظم أشكال ا

In [24]:
df = pd.DataFrame(filtered_data)


df.to_csv('KPIs.csv', index=False)

In [25]:

# # Load the JSON data
# with open('KPI.json', 'r') as f:
#     data = json.load(f)


# df = pd.DataFrame(data)


# df.to_csv('KPIs.csv', index=False)

# print("JSON data has been converted to CSV successfully!")
